# Experimento 7
***
- Conjunto de Dados: VinBigData
- Testando a predição por multi-classe
- Arquitetura utilizada: ResNet

### Importação dos pacotes

In [4]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from keras.applications.resnet import ResNet152
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint

import warnings
warnings.filterwarnings("ignore")

### Pré-processamento nos dados

In [5]:
# lendo os dados de um arquivo csv
dataframe = pd.read_csv('/content/drive/MyDrive/vinbigdata/train.csv')
# criando uma coluna com os caminhos relativos as imagens
dataframe['image_path'] = '/content/drive/MyDrive/vinbigdata/train/' + dataframe.image_id + '.jpg'

In [6]:
print('total de imagens disponíveis:', str(len(set(dataframe['image_path']))))

total de imagens disponíveis: 15000


In [7]:
# visualizando os casos disponíveis
dataframe['class_name'].value_counts()

No finding            31818
Aortic enlargement     7162
Cardiomegaly           5427
Pleural thickening     4842
Pulmonary fibrosis     4655
Nodule/Mass            2580
Lung Opacity           2483
Pleural effusion       2476
Other lesion           2203
Infiltration           1247
ILD                    1000
Calcification           960
Consolidation           556
Atelectasis             279
Pneumothorax            226
Name: class_name, dtype: int64

In [8]:
# removendo os casos não relativos a distúrbios pulmonares
dataframe = dataframe[dataframe.class_name != 'Aortic enlargement']
dataframe = dataframe[dataframe.class_name != 'Cardiomegaly']
dataframe = dataframe[dataframe.class_name != 'Other lesion']

In [9]:
# visualizando os casos disponíveis
dataframe['class_name'].value_counts()

No finding            31818
Pleural thickening     4842
Pulmonary fibrosis     4655
Nodule/Mass            2580
Lung Opacity           2483
Pleural effusion       2476
Infiltration           1247
ILD                    1000
Calcification           960
Consolidation           556
Atelectasis             279
Pneumothorax            226
Name: class_name, dtype: int64

In [10]:
# separando os casos rotulados como normais e anormais
normal_cases = dataframe[(dataframe.class_id == 14) & (dataframe.class_name == 'No finding')]
abnormal_cases = dataframe[(dataframe.class_id != 14) & (dataframe.class_name != 'No finding')]

print('total de dados após a filtração:', str(len(set(normal_cases['image_path'])) + len(set(abnormal_cases['image_path']))))

total de dados após a filtração: 13952


In [11]:
# removendo as imagens repetidas
normal_data = normal_cases[['image_path', 'class_name']].drop_duplicates(subset = 'image_path', )
abnormal_data = abnormal_cases[['image_path', 'class_name']].drop_duplicates(subset = 'image_path', )

# criando dataframes especifos com caminhos para as imagens e rótulos
normal_data['target'] = 'normal'
abnormal_data['target'] = 'abnormal'

In [12]:
# visualizando a quantidade de exemplos após a remoção de duplicatas
abnormal_data['class_name'].value_counts()

Pleural thickening    901
Pulmonary fibrosis    742
Lung Opacity          427
Nodule/Mass           339
Pleural effusion      328
Calcification         167
Infiltration          163
ILD                   152
Consolidation          59
Atelectasis            41
Pneumothorax           27
Name: class_name, dtype: int64

In [13]:
print('quantidade de dados rotulados como normais:', len(normal_data))
print('quantidade de dados rotulados como anormais:', len(abnormal_data))

quantidade de dados rotulados como normais: 10606
quantidade de dados rotulados como anormais: 3346


In [14]:
# removendo 69% dos casos normais para balancear os dados
normal, _ = train_test_split(normal_data, test_size = 0.90, random_state = 42)

In [15]:
print('quantidade de dados rotulados como normais:', len(normal))
print('quantidade de dados rotulados como anormais:', len(abnormal_data))

quantidade de dados rotulados como normais: 1060
quantidade de dados rotulados como anormais: 3346


In [16]:
# concatenando os dataframes de casos normais e anormais
full_data = pd.concat([normal, abnormal_data])

In [17]:
# misturando todos os dados do dataframe e reiniciando os valores dos índices 
full_data = full_data.sample(frac = 1, axis = 0, random_state = 42).reset_index(drop=True)

In [18]:
# separando os dados de treinamento e de teste
train_df, test_df = train_test_split(full_data, stratify = full_data['target'],
                                     test_size = 0.2, random_state = 42)

In [19]:
# separando os dados de validação dos dados de treinamento
train_df, validation_df = train_test_split(train_df, stratify = train_df['target'],
                                           test_size = 0.2, random_state = 42)

In [20]:
# visualizando a quantidade de dados
print('quantidade de imagens de treinamento:', len(train_df['image_path']))
print('quantidade de rótulos de treinamento:', len(train_df['class_name']))
print('quantidade de imagens de teste:', len(test_df['image_path']))
print('quantidade de rótulos de teste:', len(test_df['class_name']))
print('quantidade de imagens de validação:', len(validation_df['image_path']))
print('quantidade de rótulos de validação:', len(validation_df['class_name']))

quantidade de imagens de treinamento: 2819
quantidade de rótulos de treinamento: 2819
quantidade de imagens de teste: 882
quantidade de rótulos de teste: 882
quantidade de imagens de validação: 705
quantidade de rótulos de validação: 705


### Aplicando Mudança de Escala Típica

In [21]:
# normalizando as imagens de treinamento e aplicando aumento de dados
image_generator = ImageDataGenerator(rescale = 1./255.,
                                     rotation_range = 10, zoom_range = 0.2)

# criando o gerador de imagens de treinamento 
train_generator = image_generator.flow_from_dataframe(
                                                      dataframe = train_df,
                                                      directory = '',
                                                      x_col = 'image_path',
                                                      y_col = 'class_name',
                                                      batch_size = 32,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'categorical',
                                                      target_size = (256, 256))

# normalizando as imagens de teste e validação
test_datagen = ImageDataGenerator(rescale = 1./255.)

# criando o gerador de imagens de validação 
valid_generator = test_datagen.flow_from_dataframe(
                                                      dataframe = validation_df,
                                                      directory = '.', 
                                                      x_col = 'image_path',
                                                      y_col = 'class_name',
                                                      batch_size = 32,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'categorical',
                                                      target_size = (256, 256))

test_generator = test_datagen.flow_from_dataframe(
                                                  dataframe = test_df, 
                                                  directory = '.',
                                                  x_col = 'image_path',
                                                  y_col = 'class_name',
                                                  batch_size = 32,
                                                  seed = 42,
                                                  shuffle = True,
                                                  class_mode = 'categorical',
                                                  target_size = (256, 256))

Found 2819 validated image filenames belonging to 12 classes.
Found 705 validated image filenames belonging to 12 classes.
Found 882 validated image filenames belonging to 12 classes.


In [22]:
# visualizando a ordem númerica das classes
train_generator.class_indices

{'Atelectasis': 0,
 'Calcification': 1,
 'Consolidation': 2,
 'ILD': 3,
 'Infiltration': 4,
 'Lung Opacity': 5,
 'No finding': 6,
 'Nodule/Mass': 7,
 'Pleural effusion': 8,
 'Pleural thickening': 9,
 'Pneumothorax': 10,
 'Pulmonary fibrosis': 11}

### Preparando a rede neural convolucional

In [23]:
# realizando a transferência de aprendizagem com a rede ResNet
base_model = ResNet152(input_shape = (256, 256, 3), weights = 'imagenet', include_top = False)

# pegando a última camada da DenseNet
x = base_model.output
# adicionando uma camada de Global Average Pooling
x = layers.GlobalAveragePooling2D()(x)
# adicionando uma camada densa com 512 neurônios
x = layers.Dense(units = 512, activation = tf.nn.relu)(x)     
# conecatando a rede uma camada com 128 neurônios e função de ativação relu
x = layers.Dense(units = 256, activation = tf.nn.relu)(x) 
# aplicando uma camada de dropout com uma taxa de 20% (normalização)
x = layers.Dropout(rate = 0.2)(x)
# adicionando a camada de saída da rede 
predictions = layers.Dense(units = 12, activation = tf.nn.softmax)(x)

# concatenando a rede importada com a rede definida (ajuste fino)
model = Model(inputs=base_model.input, outputs=predictions)

234700800/234698864 [==============================] - 4s 0us/step


In [24]:
# definindo as flags iniciais  
model.trainable = True
set_trainable = False

# para a arquitetura DenseNet, a rede será retreinada a partir da camada 'conv5_block1_1_conv'
for layer in model.layers:
    if layer.name == 'conv5_block1_1_conv':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [25]:
# compilando a rede 
model.compile(optimizer = optimizers.RMSprop(learning_rate = 0.0001), loss = 'categorical_crossentropy', 
              metrics = ['acc'])

In [26]:
# definindo o caminho pelo qual os pesos serão armazenados 
filepath = "/content/drive/MyDrive/experimentos/v2.0-exp7-ds4/resnet/transferlearning_weights.hdf5"
# callback para salvar o melhor valor dos pesos em relação ao desempenho com os dados de validação 
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, 
                             mode = 'max')

In [27]:
# definindo um array de callbacks
callbacks = [checkpoint]

In [ ]:
# treinando a rede neural convolucional
history = model.fit_generator(train_generator, steps_per_epoch = 2819 // 32, 
                              validation_data = valid_generator, validation_steps = 705 // 32,
                              callbacks = callbacks, epochs = 50, use_multiprocessing = True,
                              workers = 8)

Epoch 1/50
88/88 [==============================] - 445s 4s/step - loss: 2.1808 - acc: 0.2279 - val_loss: 2.2930 - val_acc: 0.2415

Epoch 00001: val_acc improved from -inf to 0.24148, saving model to /content/drive/MyDrive/experimentos/v2.0-exp7-ds4/resnet/transferlearning_weights.hdf5
Epoch 2/50
88/88 [==============================] - 92s 966ms/step - loss: 2.0497 - acc: 0.2838 - val_loss: 2.1107 - val_acc: 0.2614

Epoch 00002: val_acc improved from 0.24148 to 0.26136, saving model to /content/drive/MyDrive/experimentos/v2.0-exp7-ds4/resnet/transferlearning_weights.hdf5
Epoch 3/50
88/88 [==============================] - 92s 966ms/step - loss: 2.0352 - acc: 0.2801 - val_loss: 2.1484 - val_acc: 0.2443

Epoch 00003: val_acc did not improve from 0.26136
Epoch 4/50
88/88 [==============================] - 91s 964ms/step - loss: 1.9825 - acc: 0.3147 - val_loss: 2.3563 - val_acc: 0.1875

Epoch 00004: val_acc did not improve from 0.26136
Epoch 5/50
88/88 [==============================] - 9